In [23]:
import pandas as pd
import numpy as np
import os

In [24]:
import sys
sys.path.append('../python')  # Ajustar según tu estructura
from database import *

In [25]:
# Inicializar
db_manager = DatabaseManager()

⚠️ Error al cargar AISC: name 'os' is not defined


NameError: name 'pd' is not defined

### Funciones

In [2]:
def _extraer_propiedades_cirsoc(perfil):
    """
    Extraer propiedades geométricas del perfil según tipo (base CIRSOC)
    
    Parámetros:
    -----------
    perfil : pandas.Series
        Fila del DataFrame con datos del perfil
    
    Returns:
    --------
    dict : Diccionario con propiedades organizadas por categoría
    """
    
    tipo_perfil = perfil['Tipo']
    
    # Inicializar diccionario de propiedades
    props = {
        'tipo': tipo_perfil,
        'basicas': {},
        'flexion': {},
        'torsion': {},
        'seccion': {},
        'centro_corte': {},
        'disponibles': []
    }
    
    # ========================================================================
    # PERFILES DOBLE T: W, M, HP, IPE, IPN, IPB, IPBl, IPBv
    # ========================================================================
    
    if tipo_perfil in ['W', 'M', 'HP', 'IPE', 'IPN', 'IPB', 'IPBl', 'IPBv']:
        
        # Propiedades básicas
        props['basicas'] = {
            'd': perfil['d'],           # Altura total (mm)
            'bf': perfil['bf'],         # Ancho de ala (mm)
            'Ag': perfil['Ag'],         # Área bruta (mm²)
            'Peso': perfil['Peso']      # Peso (kg/m)
        }
        
        # Propiedades de flexión
        props['flexion'] = {
            'Ix': perfil['Ix'],         # Momento de inercia eje X (mm⁴)
            'Sx': perfil['Sx'],         # Módulo elástico X (mm³)
            'rx': perfil['rx'],         # Radio de giro X (mm)
            'Zx': perfil['Zx'],         # Módulo plástico X (mm³)
            'Iy': perfil['Iy'],         # Momento de inercia eje Y (mm⁴)
            'Sy': perfil['Sy'],         # Módulo elástico Y (mm³)
            'ry': perfil['ry'],         # Radio de giro Y (mm)
            'Zy': perfil['Zy']          # Módulo plástico Y (mm³)
        }
        
        # Propiedades torsionales
        props['torsion'] = {
            'J': perfil['J'],           # Constante de torsión (mm⁴)
            'Cw': perfil['Cw']          # Constante de alabeo (mm⁶)
        }
        
        # Propiedades de sección transversal
        props['seccion'] = {
            'tf': perfil['tf'],         # Espesor de ala (mm)
            'tw': perfil['tw'],         # Espesor de alma (mm) - usar tw o tw-r1
            'hw': perfil['hw'],         # Altura libre de alma (mm)
            'bf_2tf': perfil['bf/2tf'], # Relación ancho/espesor ala
            'hw_tw': perfil['hw/tw']    # Relación altura/espesor alma
        }
        
        # Para IPB, IPBl tienen radio de acuerdo
        if tipo_perfil in ['IPB', 'IPBl']:
            props['seccion']['r1'] = perfil.get('r1', None)  # Radio de acuerdo
            props['seccion']['r2'] = perfil.get('r2', None)  # Radio de acuerdo
        
        # Para IPN, UPN tienen radios
        if tipo_perfil in ['IPN']:
            props['seccion']['r1'] = perfil.get('r1', None)
            props['seccion']['r2'] = perfil.get('r2', None)
        
        # Centro de corte (en doble T coincide con centroide)
        props['centro_corte'] = {
            'xo': 0,                    # Doble T: centro corte = centroide
            'yo': 0,
            'ro': np.sqrt((perfil['Ix'] + perfil['Iy']) / perfil['Ag'])
        }
        
        # Límites para pandeo local
        if 'Lp_ala' in perfil and pd.notna(perfil.get('Lp_ala')):
            props['limites_pandeo'] = {
                'lambda_p_ala': perfil.get('Lp_ala', None),
                'lambda_r_ala': perfil.get('Lr_ala', None),
                'lambda_p_alma': perfil.get('Lp_alma', None),
                'lambda_r_alma': perfil.get('Lr_alma', None)
            }
        
        props['disponibles'] = ['d', 'bf', 'tf', 'tw', 'hw', 'Ag', 'Ix', 'Iy', 
                               'rx', 'ry', 'J', 'Cw', 'Zx', 'Zy']
    
    # ========================================================================
    # PERFILES CANAL: C, MC, UPN
    # ========================================================================
    
    elif tipo_perfil in ['C', 'MC', 'UPN']:
        
        # Propiedades básicas
        props['basicas'] = {
            'd': perfil['d'],
            'bf': perfil['bf'],
            'Ag': perfil['Ag'],
            'Peso': perfil['Peso']
        }
        
        # Propiedades de flexión
        props['flexion'] = {
            'Ix': perfil['Ix'],
            'Sx': perfil['Sx'],
            'rx': perfil['rx'],
            'Zx': perfil['Zx'],
            'Iy': perfil['Iy'],
            'Sy': perfil['Sy'],
            'ry': perfil['ry'],
            'Zy': perfil['Zy']
        }
        
        # Propiedades torsionales
        props['torsion'] = {
            'J': perfil['J'],
            'Cw': perfil['Cw']
        }
        
        # Propiedades de sección transversal
        props['seccion'] = {
            'tf': perfil['tf'],
            'tw': perfil['tw'],
            'hw': perfil['hw'],
            'bf_2tf': perfil['bf/2tf'],
            'hw_tw': perfil['hw/tw']
        }
        
        # Para UPN tienen radios y propiedades adicionales
        if tipo_perfil == 'UPN':
            props['seccion']['r1'] = perfil.get('r1', None)
            props['seccion']['r2'] = perfil.get('r2', None)
        
        # Centro de corte (DESPLAZADO en canales)
        # UPN tiene columnas específicas para centro de corte
        if tipo_perfil == 'UPN':
            props['centro_corte'] = {
                'xo': perfil.get('x', 0) if pd.notna(perfil.get('x')) else 0,  # Distancia centro corte
                'eo': perfil.get('eo', 0) if pd.notna(perfil.get('eo')) else 0,
                'yo': 0,  # En canal, desplazamiento solo en X
                'ro': None  # Se calculará después
            }
        elif tipo_perfil in ['C', 'MC']:
            # Para C y MC, usar columnas x, eo si existen
            props['centro_corte'] = {
                'xo': perfil.get('x', 0) if pd.notna(perfil.get('x')) else 0,
                'eo': perfil.get('eo', 0) if pd.notna(perfil.get('eo')) else 0,
                'yo': 0,
                'ro': None
            }
        
        # Calcular ro
        xo = props['centro_corte']['xo']
        yo = props['centro_corte']['yo']
        Ix = props['flexion']['Ix']
        Iy = props['flexion']['Iy']
        Ag = props['basicas']['Ag']
        props['centro_corte']['ro'] = np.sqrt(xo**2 + yo**2 + (Ix + Iy) / Ag)
        
        # Límites para pandeo local
        if tipo_perfil == 'UPN' and 'Lp_ala' in perfil:
            props['limites_pandeo'] = {
                'lambda_p_ala': perfil.get('Lp_ala', None),
                'lambda_r_ala': perfil.get('Lr_ala', None),
                'lambda_p_alma': perfil.get('Lp_alma', None),
                'lambda_r_alma': perfil.get('Lr_alma', None)
            }
        elif tipo_perfil in ['C', 'MC']:
            props['limites_pandeo'] = {
                'lambda_p_ala': perfil.get('Lp_ala', None),
                'lambda_r_ala': perfil.get('Lr_ala', None),
                'lambda_p_alma': perfil.get('Lp_alma', None),
                'lambda_r_alma': perfil.get('Lr_alma', None)
            }
        
        props['disponibles'] = ['d', 'bf', 'tf', 'tw', 'hw', 'Ag', 'Ix', 'Iy', 
                               'rx', 'ry', 'J', 'Cw', 'x', 'eo']
    
    # ========================================================================
    # PERFILES ANGULARES: L
    # ========================================================================
    
    elif tipo_perfil == 'L':
        
        # Los ángulos tienen una estructura diferente
        props['basicas'] = {
            'Designacion': perfil['Designacion'],  # Ej: L100x100x10
            'b': perfil['b'],                      # Ancho del ala (mm)
            't': perfil['t'],                      # Espesor (mm)
            'Ag': perfil['Ag'],                    # Área bruta (mm²)
            'Peso': perfil['Peso']                 # Peso (kg/m)
        }
        
        # Propiedades de flexión (ejes principales rotados)
        # Los ángulos tienen Ix-Iy (eje mayor y menor)
        props['flexion'] = {
            'Ix': perfil['Ix-Iy'],      # Momento de inercia eje mayor (mm⁴)
            'Sx': perfil['Sx-Sy'],      # Módulo elástico eje mayor (mm³)
            'rx': perfil['rx-ry'],      # Radio de giro eje mayor (mm)
            'Iy': perfil['Ix-Iy'],      # Momento de inercia eje menor (mm⁴) - mismo valor
            'Sy': perfil['Sx-Sy'],      # Módulo elástico eje menor (mm³)
            'ry': perfil['rx-ry'],      # Radio de giro eje menor (mm)
            'Iv': perfil['Iv'],         # Momento de inercia eje v (mm⁴)
            'Sv': perfil['Sv'],         # Módulo elástico eje v (mm³)
            'iv': perfil['iv'],         # Radio de giro eje v (mm)
            'Iz': perfil['Iz'],         # Momento de inercia eje z (mm⁴)
            'iz': perfil['iz']          # Radio de giro eje z (mm)
        }
        
        # Propiedades torsionales
        props['torsion'] = {
            'J': perfil['J'],
            'Cw': perfil['Cw']
        }
        
        # Propiedades de sección transversal
        props['seccion'] = {
            'b': perfil['b'],           # Ancho de ala
            't': perfil['t'],           # Espesor
            'r': perfil.get('r', None), # Radio de acuerdo interior
            'r1': perfil.get('r1', None),  # Radio de acuerdo exterior
            'b_t': perfil['b/t']        # Relación ancho/espesor
        }
        
        # Excentricidades del centroide
        props['centro_corte'] = {
            'ex': perfil['ex-ey'],      # Excentricidad en x
            'ey': perfil['ex-ey'],      # Excentricidad en y (mismo valor para L iguales)
            'xo': perfil['ex-ey'],      # Para cálculos de pandeo
            'yo': perfil['ex-ey'],
            'ro': None  # Se calculará
        }
        
        # Calcular ro para ángulos
        ex = props['centro_corte']['ex']
        Ag = props['basicas']['Ag']
        # Para ángulos, aproximación
        Ix_est = perfil['Ix-Iy']
        Iy_est = perfil['Ix-Iy']
        props['centro_corte']['ro'] = np.sqrt(ex**2 + ex**2 + (Ix_est + Iy_est) / Ag)
        
        props['disponibles'] = ['b', 't', 'Ag', 'Ix-Iy', 'Sx-Sy', 'rx-ry', 
                               'Iv', 'Sv', 'iv', 'J', 'Cw', 'ex-ey']
    
    # ========================================================================
    # TIPO NO RECONOCIDO
    # ========================================================================
    
    else:
        raise ValueError(f"Tipo de perfil '{tipo_perfil}' no soportado en base CIRSOC")
    
    return props


# ============================================================================
# FUNCIÓN AUXILIAR: Verificar disponibilidad de propiedades
# ============================================================================

def _verificar_propiedades_disponibles(props, tipo_perfil):
    """
    Verificar qué propiedades están disponibles y cuáles faltan
    
    Returns:
    --------
    dict : {
        'completo': bool,
        'faltantes': list,
        'advertencias': list
    }
    """
    
    verificacion = {
        'completo': True,
        'faltantes': [],
        'advertencias': []
    }
    
    # Verificar propiedades críticas según tipo
    if tipo_perfil in ['W', 'M', 'HP', 'IPE', 'IPN', 'IPB', 'IPBl', 'IPBv']:
        # Doble T
        requeridas = ['d', 'bf', 'tf', 'tw', 'hw', 'Ag', 'Ix', 'Iy', 'rx', 'ry']
        opcionales_importantes = ['J', 'Cw']
        
    elif tipo_perfil in ['C', 'MC', 'UPN']:
        # Canal
        requeridas = ['d', 'bf', 'tf', 'tw', 'hw', 'Ag', 'Ix', 'Iy', 'rx', 'ry']
        opcionales_importantes = ['J', 'Cw', 'x', 'eo']
        
    elif tipo_perfil == 'L':
        # Ángulo
        requeridas = ['b', 't', 'Ag', 'Ix-Iy', 'rx-ry']
        opcionales_importantes = ['J', 'Cw', 'ex-ey']
        
    else:
        return verificacion
    
    # Verificar propiedades requeridas
    for prop in requeridas:
        if prop not in props['disponibles']:
            verificacion['faltantes'].append(prop)
            verificacion['completo'] = False
    
    # Verificar propiedades opcionales importantes
    for prop in opcionales_importantes:
        if prop not in props['disponibles']:
            verificacion['advertencias'].append(
                f"Propiedad '{prop}' no disponible - puede limitar el cálculo"
            )
    
    return verificacion

Paso a paso del algoritmo

1. Determinar tipo de sección (doble T, canal, ángulo, etc.)

2. Para cada elemento de la sección:
   - Calcular λ = b/t (o h/tw, o D/t según corresponda)
   - Comparar con λp y λr
   - Clasificar: Compacta, No compacta, o Esbelta

3. Clasificar la sección completa:
   - Si TODOS los elementos son compactos → Sección COMPACTA
   - Si ALGÚN elemento es esbelta → Sección ESBELTA
   - Caso contrario → Sección NO COMPACTA

4. Calcular resistencia:

   SI sección es COMPACTA o NO COMPACTA:
       A = Ag (área bruta completa)
       
   SI sección es ESBELTA:
       Calcular Ae (área efectiva reducida) - proceso iterativo
       A = Ae

5. Calcular pandeo global (flexional, torsional, flexo-torsional):
   
   Fe = π²E/(KL/r)²  (usar radio de giro apropiado)
   
   Para secciones esbeltas con factor Q:
       Si Q·Fy/Fe ≤ 2.25:
           Fcr = Q·[0.658^(Q·Fy/Fe)]·Fy
       Si Q·Fy/Fe > 2.25:
           Fcr = 0.877·Fe
           
   Para secciones no esbeltas:
       Si Fy/Fe ≤ 2.25:
           Fcr = [0.658^(Fy/Fe)]·Fy
       Si Fy/Fe > 2.25:
           Fcr = 0.877·Fe

6. Calcular resistencia nominal y de diseño:
   
   Pn = Fcr · A  (donde A = Ag o Ae según corresponda)
   Pd = φc · Pn = 0.90 · Fcr · A

#### Desarrollo

In [19]:
# ============================================================================
# ARCHIVO: compresion.py
# CÁLCULO DE RESISTENCIA A COMPRESIÓN SEGÚN AISC 360
# ============================================================================

import numpy as np
import pandas as pd


# ============================================================================
# INICIALIZACIÓN DE BASE DE DATOS
# ============================================================================

db_manager = DatabaseManager()
db_manager.set_database('CIRSOC')

⚠️ Error al cargar AISC: name 'os' is not defined


NameError: name 'pd' is not defined

In [ ]:
def compresion(perfil_nombre, Fy, Lx, Ly, Lz=None, Kx=1.0, Ky=1.0, Kz=1.0, mostrar_calculo=True):
    
        """
    Calcular resistencia a compresión axial según AISC 360
    
    Parámetros:
    -----------
    perfil_nombre : str
        Nombre del perfil según base de datos (ej: 'W310X97')
    Fy : float
        Tensión de fluencia del acero (MPa si SI, ksi si Imperial)
    Lx : float
        Longitud no arriostrada en dirección X (mm si SI, in si Imperial)
    Ly : float
        Longitud no arriostrada en dirección Y (mm si SI, in si Imperial)
    Lz : float, optional
        Longitud no arriostrada para torsión (mm si SI, in si Imperial)
        Si None, se asume igual a max(Lx, Ly)
    Kx : float, default=1.0
        Factor de longitud efectiva para eje X
    Ky : float, default=1.0
        Factor de longitud efectiva para eje Y
    Kz : float, default=1.0
        Factor de longitud efectiva para torsión
    mostrar_calculo : bool, default=True
        Si True, genera reporte detallado del cálculo
    
    Returns:
    --------
    dict : Diccionario con resultados y reporte
        {
            'Pn': Resistencia nominal (kN o kips),
            'Pd': Resistencia de diseño (kN o kips),
            'phi_c': Factor de resistencia (0.90),
            'Fcr': Tensión crítica de pandeo (MPa o ksi),
            'modo_critico': str describiendo el modo de pandeo crítico,
            'lambda_x': Esbeltez efectiva en X,
            'lambda_y': Esbeltez efectiva en Y,
            'clasificacion': str ('Compacta', 'No Compacta', 'Esbelta'),
            'A_efectiva': Área efectiva usada (mm² o in²),
            'factor_Q': Factor de reducción por pandeo local,
            'verificaciones': dict con verificaciones de pandeo local,
            'reporte': str con cálculo detallado formateado,
            'advertencias': list de advertencias/notas importantes
        }
    
    Raises:
    -------
    ValueError : Si el perfil no existe o faltan propiedades necesarias
    """
    
     # ========================================================================
    # CONSTANTES DEL SISTEMA (SIEMPRE SI)
    # ========================================================================
    
    E = 200000      # Módulo de elasticidad en MPa
    G = 77200       # Módulo de corte en MPa
    phi_c = 0.90    # Factor de resistencia a compresión
    
    # ========================================================================
    # PASO 1: OBTENER DATOS DEL PERFIL
    # ========================================================================
    
    # Determino de que sección se trata para todas las posibles de ambas bases de datos, dividiendo entre sección con dos ejes de simetria, una sola, o ninguna
    try:
        perfil = db_manager.get_perfil_data(perfil_nombre)
    except Exception as e:
        raise ValueError(f"Error al obtener perfil '{perfil_nombre}': {e}")
    
    # Verificar que el perfil esté en la base activa
    db_name = db_manager.get_database_name()
    
    #Determino familia
    tipo_perfil = perfil['Tipo']
    
    if tipo_perfil in ['HP', 'IPB', 'IPBl', 'IPBv', 'IPE', 'IPN', 'M', 'W']:
        simetria = 'doble'
        
    elif tipo_perfil in ['C', 'MC', 'UPN']:
        simetria = 'simple'
        
    elif tipo_perfil in ['L']:
        simetria = 'sin_simetria'
    else:
        print('La familia no esta categorizada. Se procede suponiendo que no tiene simetria')
        simetria = 'sin_simetria'
    
    
    # ========================================================================
    # PASO 2: EXTRAER PROPIEDADES GEOMÉTRICAS DEL PERFIL (CIRSOC)
    # ========================================================================
    
    # Extraer propiedades según tipo de perfil
    props = _extraer_propiedades_cirsoc(perfil)
    
    # Verificar disponibilidad
    verificacion = _verificar_propiedades_disponibles(props, tipo_perfil)
    
    if not verificacion['completo']:
        raise ValueError(
            f"Perfil '{perfil_nombre}' no tiene todas las propiedades necesarias.\n"
            f"Propiedades faltantes: {', '.join(verificacion['faltantes'])}"
        )
    
    # Agregar advertencias al resultado
    resultados['advertencias'].extend(verificacion['advertencias'])
    
    # Asignar propiedades a variables de trabajo
    # PROPIEDADES BÁSICAS
    A = props['basicas']['Ag']              # Área bruta en mm²
    d = props['basicas']['d'] if 'd' in props['basicas'] else props['basicas'].get('b', None)
    
    # PROPIEDADES DE FLEXIÓN
    Ix = props['flexion']['Ix']             # mm⁴
    Iy = props['flexion']['Iy']             # mm⁴
    rx = props['flexion']['rx']             # mm
    ry = props['flexion']['ry']             # mm
    
    # PROPIEDADES TORSIONALES
    J = props['torsion'].get('J', 0)        # mm⁴
    Cw = props['torsion'].get('Cw', 0)      # mm⁶
    
    # PROPIEDADES DE SECCIÓN (para pandeo local)
    if tipo_perfil in ['W', 'M', 'HP', 'IPE', 'IPN', 'IPB', 'IPBl', 'IPBv', 'C', 'MC', 'UPN']:
        bf = props['basicas']['bf']         # mm
        tf = props['seccion']['tf']         # mm
        tw = props['seccion']['tw']         # mm
        hw = props['seccion']['hw']         # mm
        bf_2tf = props['seccion']['bf_2tf'] # Relación ya calculada
        hw_tw = props['seccion']['hw_tw']   # Relación ya calculada
        
    elif tipo_perfil == 'L':
        b = props['basicas']['b']           # mm
        t = props['seccion']['t']           # mm
        b_t = props['seccion']['b_t']       # Relación ya calculada
        bf = b  # Para consistencia en el código
        tf = t
        tw = t
        hw = None
    
    # CENTRO DE CORTE
    xo = props['centro_corte'].get('xo', 0)     # mm
    yo = props['centro_corte'].get('yo', 0)     # mm
    ro = props['centro_corte'].get('ro', None)  # mm
    
    if ro is None:
        ro = np.sqrt(xo**2 + yo**2 + (Ix + Iy)/A)
    
    # Guardar propiedades en resultados para referencia
    resultados['propiedades'] = props
    
    
     # ========================================================================
    # PASO 3: AJUSTAR Lz SI NO SE PROPORCIONÓ
    # ========================================================================
    
    if Lz is None:
        Lz = max(Lx, Ly)
    
    # ========================================================================
    # PASO 4: INICIALIZAR DICCIONARIO DE RESULTADOS
    # ========================================================================
    
    resultados = {
        'perfil': perfil_nombre,
        'tipo': tipo_perfil,
        'base_datos': db_name,
        'Fy': Fy,
        'E': E,
        'G': G,
        'phi_c': phi_c,
        'A': A,
        'rx': rx,
        'ry': ry,
        'Lx': Lx,
        'Ly': Ly,
        'Lz': Lz,
        'Kx': Kx,
        'Ky': Ky,
        'Kz': Kz,
        'advertencias': []
    }
    
    # ========================================================================
    # PASO 5: IDENTIFICAR FAMILIA DE PERFIL
    # ========================================================================
    
    if tipo_perfil in ['W', 'S', 'M', 'HP', 'IPE', 'IPN', 'IPB', 'IPBv', 'IPBl']:
        familia = 'DOBLE_T'
        num_ejes_simetria = 2
        tiene_centro_corte_desplazado = False
        
    elif tipo_perfil in ['C', 'MC', 'UPN']:
        familia = 'CANAL'
        num_ejes_simetria = 1
        tiene_centro_corte_desplazado = True
        
    elif tipo_perfil == 'HSS' or 'HSS' in tipo_perfil:
        if D_hss is not None and D_hss > 0:
            familia = 'HSS_CIRCULAR'
            num_ejes_simetria = 'infinito'
            tiene_centro_corte_desplazado = False
        else:
            familia = 'HSS_RECTANGULAR'
            num_ejes_simetria = 2
            tiene_centro_corte_desplazado = False
            
    elif tipo_perfil in ['WT', 'ST', 'MT']:
        familia = 'TEE'
        num_ejes_simetria = 1
        tiene_centro_corte_desplazado = True
        
    else:
        familia = 'GENERAL'
        num_ejes_simetria = 0
        tiene_centro_corte_desplazado = True
        resultados['advertencias'].append(
            f"Tipo de perfil '{tipo_perfil}' no reconocido. Se usará análisis general."
        )
    
    resultados['familia'] = familia
    resultados['num_ejes_simetria'] = num_ejes_simetria
    
    # ========================================================================
    # PASO 6: VERIFICAR DISPONIBILIDAD DE PROPIEDADES CRÍTICAS
    # ========================================================================
    
    # Para pandeo torsional y flexo-torsional se necesitan J y Cw
    if (familia in ['CANAL', 'TEE', 'GENERAL']) and (J == 0 or Cw == 0):
        resultados['advertencias'].append(
            "Propiedades torsionales (J, Cw) no disponibles. "
            "No se verificará pandeo torsional/flexo-torsional."
        )
        verificar_torsion = False
    else:
        verificar_torsion = True
    
    # Para pandeo local se necesitan espesores
    if familia == 'DOBLE_T' and (bf is None or tf is None or tw is None):
        resultados['advertencias'].append(
            "Propiedades de sección (bf, tf, tw) no disponibles completamente. "
            "Verificación de pandeo local limitada."
        )
    
    # ========================================================================
    # AQUÍ CONTINÚA EL ALGORITMO DE CÁLCULO
    # ========================================================================
    
    # [PASO 7: Verificación de pandeo local]
    # [PASO 8: Cálculo de pandeo global]
    # [PASO 9: Determinación de Fcr y Pn]
    # [PASO 10: Generación de reporte]
    
    return resultados
    
    # Calculo los gamma que corresponda para el caso de columna flexocomprimida
    
    
    # Defino que tipo de sección es
    
    # Si sección es compacta determino el valor de A
    
    # Si sección es esbelta, calculo Ae
    
    
    # En función del tipo de simetría,
    
    ### DOBLEMENTE SIMETRICA
    
    #### PANDEO FLEXIONAL ALREDEDOR DE EJE FUERTE
    
    #### PANDEO FLEXIONAL PURO ALREDEDOR DE EJE DEBIL
    
    #### PANDEO TORSIONAL PURO
    
    ### UN EJE DE SIMETRIA
    
    #### PANDEO FLEXIONAL PURO ALREDEDOR DE EJE DE SIMETRIA
    
    
    #### PANDEO FLEXOTORSIONAL
    
    
    ### SIN EJES DE SIMETRIA
    
    #### PANDEO FLEXOTORSIONAL
    
    
    
    